# Decorator introduction

##  Closure
**First-class function is the function that can be treated as any other objects, to be passed as an argument, or returned by another function, or assigned to a variable...**

**Close is a nested functions, which returns one First-class function**

In [1]:
from JL_tools import print_title

In [2]:
# Closure example

def outer_function(msg):
    message=msg
    
    def inner_function():
        print(message)
    return inner_function

hi_func = outer_function('Hi')
bye_func = outer_function('Bye')

print_title('Closure')
hi_func()
bye_func()

--------------------Closure--------------------
Hi
Bye


## Decorator function

Decorator is a convenient ''Wrapper'', which take another function as an argument, adding some more functionality without modifying the original function.

In [3]:
# Decorator function return the wrapped function, which is waiting to be executed

def decorator_function(original_function):
    def wrapper_function():
        return original_function()
    return wrapper_function

# A simple example of argument function
def display():
    print('display function ran')
    
print_title('Decorator function')  
decorated_display = decorator_function(display)

decorated_display()

--------------------Decorator function--------------------
display function ran


Now, Let's add some more functionalities!

In [4]:
def decorator_function(original_function):
    def wrapper_function():
        print('wrapper executed this before {}'.format(original_function.__name__))  ## Add some extra output
        return original_function()
    return wrapper_function

print_title('Decorator function')    
decorated_display = decorator_function(display)

decorated_display()

--------------------Decorator function--------------------
wrapper executed this before display
display function ran


## now is the Decorator

It is an easier way to realized the same complicated functionality above.

```
@decorator_function
def display2():
    print('display function ran again')

display2()
    
putting @decorator_function right above the function to be decorated, is exactly the same as:
decorated_display2 = decorator_function(display2)
decorated_display2()
```

In [5]:
# A Decorator
@decorator_function
def display2():
    print('display function ran again')
    
display2()
    
# putting @decorator_function right above the function to be decorated, is exactly the same as:
# decorated_display2 = decorator_function(display2)
# decorated_display2()

wrapper executed this before display2
display function ran again


In [6]:
# If the wrapper function takes arbitrary number of arguments
def decorator_function2(original_function):
    def wrapper_function(*args, **kwargs):
        print('wrapper executed this before {}'.format(original_function.__name__))  ## Add some extra output
        return original_function(*args, **kwargs)
    return wrapper_function

@decorator_function2
def display_info(name, age):
    print('display function ran with ({}, {})'.format(name,age))

print_title('wrapped function takes arbitrary number of arguments')
display_info('John',25)

--------------------wrapped function takes arbitrary number of arguments--------------------
wrapper executed this before display_info
display function ran with (John, 25)


In [7]:
# Using class to create Decorator has the same effect

# If the wrapper function takes arbitrary number of arguments
from functools import wraps

def decorator_function(original_function):
    @wraps(original_function)  # This is to let the wrapped function display its own name
    def wrapper_function(*args, **kwargs):
        print('wrapper executed this before {}'.format(original_function.__name__))
        return original_function(*args, **kwargs)
    return wrapper_function

  
# Using decorator class, the same effect
class decorator_class():
    def __init__(self, original_function):
        self.original_function = original_function

    def __call__(self, *args, **kwargs):
        print('call method executed this before {}'.format(self.original_function.__name__))
        return self.original_function(*args, **kwargs)

@decorator_function
def display():
    print('display function ran')

print_title('Decorator class has the same effect as Decorator function')
@decorator_class
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display()
print('Function name is {}'.format(display.__name__))

display_info('John',25)

--------------------Decorator class has the same effect as Decorator function--------------------
wrapper executed this before display
display function ran
Function name is display
call method executed this before display_info
display_info ran with arguments (John, 25)


## @wraps

@wraps is a convenient decorator to fix the name of the wrapped function and nested wrapper

putting double or even more decorators, such as
```
@my_timer
@my_logger  # without wraps in the wrappers, this will be wrong
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))
```

is the same as
```
my_display=my_timer(my_logger(display_info)) 
```

In [8]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

#     @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info('Ran with args: {}, and kwargs: {}'.format(args,kwargs))
        return orig_func(*args, **kwargs)
    return wrapper


def my_timer(orig_func):
    import time

#     @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time()-t1
        print('{} ran in : {} sec'.format(orig_func.__name__, t2))
        return result
    return wrapper

@my_timer
@my_logger  # without wraps in the wrappers, this will be wrong
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

print_title('using @wraps generate the correct __name__ and functionality of nested decorators')
display_info('John',25)
print('Function name is {}'.format(display_info.__name__))

# my_display=my_timer(my_logger(display_info))  # Actually the wrapper function of my_logger is passed to my_timer
# my_display('John',25)
# print(my_display.__name__)

--------------------using @wraps generate the correct __name__ and functionality of nested decorators--------------------
display_info ran with arguments (John, 25)
wrapper ran in : 0.0 sec
Function name is wrapper


In [9]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info('Ran with args: {}, and kwargs: {}'.format(args,kwargs))
        return orig_func(*args, **kwargs)
    return wrapper


def my_timer(orig_func):
    import time

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time()-t1
        print('{} ran in : {} sec'.format(orig_func.__name__, t2))
        return result
    return wrapper

@my_timer
@my_logger  # without wraps in the wrappers, this will be wrong
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

print_title('using @wraps generate the correct __name__ and functionality of nested decorators')
display_info('John',25)
print('Function name is {}'.format(display_info.__name__))

# my_display=my_timer(my_logger(display_info))  # Actually the wrapper function of my_logger is passed to my_timer
# my_display('John',25)
# print(my_display.__name__)

--------------------using @wraps generate the correct __name__ and functionality of nested decorators--------------------
display_info ran with arguments (John, 25)
display_info ran in : 0.0 sec
Function name is display_info
